In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold


In [2]:
data = pd.read_csv('ISTrend_data.csv')
data

,Company,Time,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Classification
0,AMZN,2019-12-01,47635.0,1983.0,1515.0,89.65,2.78,2.89,0.00,2073.0,1983.0,1532.5,1
1,TSCO,2018-12-01,654.6,15.3,109.8,7.63,1.03,0.19,0.15,15.3,15.3,6.8,1
2,AZO,2019-08-01,642.6,81.8,279.7,59.14,15.19,5.43,0.00,80.9,81.8,44.7,1
3,EQT,2018-12-01,1934.8,221.7,-3753.1,3.36,-16.65,0.06,0.00,250.6,221.7,106.1,0
4,SOHU,2013-12-01,333.1,-57.8,-88.2,8.54,-2.32,-2.31,0.00,-45.1,-57.8,-88.1,1
5,DRI,2019-05-01,430.3,81.3,117.4,3.75,0.97,0.47,0.48,81.7,81.3,56.2,1
6,PH,2017-06-01,668.5,166.3,176.6,6.26,1.41,0.74,0.06,202.1,166.3,87.5,1
7,VRTX,2011-12-01,1267.2,916.3,784.2,6.16,3.91,2.81,0.00,916.3,916.3,564.3,0
8,PFG,2014-12-01,1188.1,513.6,231.4,4.02,0.71,0.76,0.30,515.5,513.6,223.7,1
9,DISH,2012-12-01,107.2,-618.2,-879.2,-0.08,-1.99,-0.74,0.00,-613.2,-618.2,-320.7,1


In [3]:
classifications = data['Classification'] 
data = data.drop(['Company', 'Time', 'Classification'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Classification', classifications)
data


Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Classification
0,0.754239,0.212925,0.488938,0.037673,0.344334,0.165383,0.480023,0.214329,0.212925,0.249985,1
1,0.553576,0.196357,0.481506,0.032382,0.344310,0.165323,0.488709,0.197003,0.196357,0.230938,1
2,0.553525,0.196917,0.482404,0.035705,0.344506,0.165439,0.480023,0.197556,0.196917,0.231412,1
3,0.559044,0.198095,0.461074,0.032107,0.344065,0.165321,0.480023,0.198985,0.198095,0.232178,0
4,0.552203,0.195741,0.480458,0.032441,0.344264,0.165268,0.480023,0.196495,0.195741,0.229754,1
5,0.552618,0.196912,0.481546,0.032132,0.344309,0.165330,0.507817,0.197562,0.196912,0.231555,1
6,0.553636,0.197628,0.481859,0.032294,0.344315,0.165336,0.483497,0.198576,0.197628,0.231946,1
7,0.556193,0.203943,0.485073,0.032287,0.344350,0.165381,0.480023,0.204590,0.203943,0.237898,0
8,0.555855,0.200552,0.482149,0.032149,0.344305,0.165336,0.497394,0.201215,0.200552,0.233646,1
9,0.551238,0.191023,0.476275,0.031885,0.344268,0.165303,0.480023,0.191711,0.191023,0.226850,1


In [4]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[0.75423917 0.21292468 0.48893807 ... 0.21432902 0.21292468 0.24998533]
 [0.55357628 0.19635667 0.48150562 ... 0.19700335 0.19635667 0.23093844]
 [0.55352502 0.1969166  0.48240426 ... 0.1975557  0.1969166  0.23141158]
 ...
 [0.5658876  0.19423231 0.47595825 ... 0.19441591 0.19423231 0.23161008]
 [0.53574144 0.17061424 0.4720971  ... 0.17126114 0.17061424 0.2105595 ]
 [0.55532448 0.19673389 0.48112056 ... 0.19751528 0.19673389 0.23125428]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 4500


[1. 1. 1. ... 1. 0. 1.]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 1.0
Total number of y values 4500


In [5]:
import tensorflow as tf
model = tf.keras.Sequential([
    layers.Dense(units=100, activation='relu', input_shape=(number_of_features,)),
    layers.Dropout(0.2),
    layers.Dense(units = 1, activation='sigmoid')
])

model.compile(
            loss = 'binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#give summary of model
model.summary()

scores = []
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x= X_train, y= y_train, batch_size= 5, epochs=1000) # Fit Model
    results = model.evaluate(x= X_test, y=y_test) # Evaluate Model Performance, Appending model accuracy to scores list
    scores.append(results[1]) # Appending model accuracy to scores list
    
scores = np.array(scores)
print(scores)
print("Mean Accuracy:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________
Train on 3599 samples
Epoch 1/1000
3599/3599 [==============================] - 3s 840us/sample - loss: 0.6333 - accuracy: 0.6752
Epoch 2/1000
3599/3599 [==============================] - 2s 450us/sample - loss: 0.6308 - accuracy: 0.6752
Epoch 3/1000
3599/3599 [==============================] - 2s 503us/sample - loss: 0.6309 - accuracy: 0.6752
Epoch 4/1000
3599/3599 [==============

3599/3599 [==============================] - 1s 397us/sample - loss: 0.6242 - accuracy: 0.6799
Epoch 144/1000
3599/3599 [==============================] - 2s 496us/sample - loss: 0.6232 - accuracy: 0.6794
Epoch 145/1000
3599/3599 [==============================] - 2s 588us/sample - loss: 0.6234 - accuracy: 0.6813
Epoch 146/1000
3599/3599 [==============================] - 2s 634us/sample - loss: 0.6246 - accuracy: 0.6799
Epoch 147/1000
3599/3599 [==============================] - 2s 574us/sample - loss: 0.6253 - accuracy: 0.6785
Epoch 148/1000
3599/3599 [==============================] - 2s 573us/sample - loss: 0.6245 - accuracy: 0.6794
Epoch 149/1000
3599/3599 [==============================] - 2s 628us/sample - loss: 0.6255 - accuracy: 0.6807
Epoch 150/1000
3599/3599 [==============================] - 2s 572us/sample - loss: 0.6246 - accuracy: 0.6794
Epoch 151/1000
3599/3599 [==============================] - 2s 620us/sample - loss: 0.6244 - accuracy: 0.6788
Epoch 152/1000
3599/3599 

3599/3599 [==============================] - 1s 358us/sample - loss: 0.6221 - accuracy: 0.6810
Epoch 292/1000
3599/3599 [==============================] - 2s 614us/sample - loss: 0.6211 - accuracy: 0.6819
Epoch 293/1000
3599/3599 [==============================] - 2s 581us/sample - loss: 0.6215 - accuracy: 0.6821
Epoch 294/1000
3599/3599 [==============================] - 1s 395us/sample - loss: 0.6209 - accuracy: 0.6824
Epoch 295/1000
3599/3599 [==============================] - 2s 680us/sample - loss: 0.6215 - accuracy: 0.6785
Epoch 296/1000
3599/3599 [==============================] - 2s 589us/sample - loss: 0.6214 - accuracy: 0.6824
Epoch 297/1000
3599/3599 [==============================] - 2s 638us/sample - loss: 0.6207 - accuracy: 0.6813
Epoch 298/1000
3599/3599 [==============================] - 2s 642us/sample - loss: 0.6209 - accuracy: 0.6827
Epoch 299/1000
3599/3599 [==============================] - 2s 571us/sample - loss: 0.6231 - accuracy: 0.6805
Epoch 300/1000
3599/3599 

3599/3599 [==============================] - 1s 344us/sample - loss: 0.6201 - accuracy: 0.6813
Epoch 440/1000
3599/3599 [==============================] - 1s 351us/sample - loss: 0.6177 - accuracy: 0.6835
Epoch 441/1000
3599/3599 [==============================] - 1s 347us/sample - loss: 0.6199 - accuracy: 0.6802
Epoch 442/1000
3599/3599 [==============================] - 2s 418us/sample - loss: 0.6178 - accuracy: 0.6796
Epoch 443/1000
3599/3599 [==============================] - 2s 561us/sample - loss: 0.6206 - accuracy: 0.6816
Epoch 444/1000
3599/3599 [==============================] - 3s 759us/sample - loss: 0.6194 - accuracy: 0.6813
Epoch 445/1000
3599/3599 [==============================] - 3s 702us/sample - loss: 0.6184 - accuracy: 0.6805
Epoch 446/1000
3599/3599 [==============================] - 2s 632us/sample - loss: 0.6176 - accuracy: 0.6824
Epoch 447/1000
3599/3599 [==============================] - 2s 666us/sample - loss: 0.6185 - accuracy: 0.6796
Epoch 448/1000
3599/3599 

3599/3599 [==============================] - 2s 603us/sample - loss: 0.6177 - accuracy: 0.6835
Epoch 588/1000
3599/3599 [==============================] - 2s 437us/sample - loss: 0.6166 - accuracy: 0.6835
Epoch 589/1000
3599/3599 [==============================] - 1s 357us/sample - loss: 0.6182 - accuracy: 0.6835
Epoch 590/1000
3599/3599 [==============================] - 2s 431us/sample - loss: 0.6171 - accuracy: 0.6816
Epoch 591/1000
3599/3599 [==============================] - 1s 379us/sample - loss: 0.6176 - accuracy: 0.6819
Epoch 592/1000
3599/3599 [==============================] - 1s 353us/sample - loss: 0.6184 - accuracy: 0.6819
Epoch 593/1000
3599/3599 [==============================] - 1s 369us/sample - loss: 0.6176 - accuracy: 0.6835
Epoch 594/1000
3599/3599 [==============================] - 2s 446us/sample - loss: 0.6167 - accuracy: 0.6821
Epoch 595/1000
3599/3599 [==============================] - 1s 372us/sample - loss: 0.6170 - accuracy: 0.6810
Epoch 596/1000
3599/3599 

...

3601/3601 [==============================] - 1s 381us/sample - loss: 0.6077 - accuracy: 0.6901
Epoch 521/1000
3601/3601 [==============================] - 1s 409us/sample - loss: 0.6092 - accuracy: 0.6873
Epoch 522/1000
3601/3601 [==============================] - 1s 376us/sample - loss: 0.6077 - accuracy: 0.6898
Epoch 523/1000
3601/3601 [==============================] - 1s 371us/sample - loss: 0.6097 - accuracy: 0.6884
Epoch 524/1000
3601/3601 [==============================] - 1s 383us/sample - loss: 0.6056 - accuracy: 0.6904
Epoch 525/1000
3601/3601 [==============================] - 2s 420us/sample - loss: 0.6062 - accuracy: 0.6926
Epoch 526/1000
3601/3601 [==============================] - 1s 397us/sample - loss: 0.6076 - accuracy: 0.6912
Epoch 527/1000
3601/3601 [==============================] - 1s 386us/sample - loss: 0.6064 - accuracy: 0.6893
Epoch 528/1000
3601/3601 [==============================] - 2s 418us/sample - loss: 0.6098 - accuracy: 0.6909
Epoch 529/1000
3601/3601 

3601/3601 [==============================] - 2s 421us/sample - loss: 0.6065 - accuracy: 0.6901
Epoch 669/1000
3601/3601 [==============================] - 1s 371us/sample - loss: 0.6108 - accuracy: 0.6859
Epoch 670/1000
3601/3601 [==============================] - 1s 378us/sample - loss: 0.6105 - accuracy: 0.6876
Epoch 671/1000
3601/3601 [==============================] - 2s 423us/sample - loss: 0.6114 - accuracy: 0.6895
Epoch 672/1000
3601/3601 [==============================] - 1s 413us/sample - loss: 0.6080 - accuracy: 0.6915
Epoch 673/1000
3601/3601 [==============================] - 1s 399us/sample - loss: 0.6071 - accuracy: 0.6893
Epoch 674/1000
3601/3601 [==============================] - 1s 393us/sample - loss: 0.6097 - accuracy: 0.6887
Epoch 675/1000
3601/3601 [==============================] - 2s 437us/sample - loss: 0.6084 - accuracy: 0.6854
Epoch 676/1000
3601/3601 [==============================] - 1s 404us/sample - loss: 0.6078 - accuracy: 0.6898
Epoch 677/1000
3601/3601 

3601/3601 [==============================] - 2s 558us/sample - loss: 0.6086 - accuracy: 0.6859
Epoch 817/1000
3601/3601 [==============================] - 2s 432us/sample - loss: 0.6075 - accuracy: 0.6904
Epoch 818/1000
3601/3601 [==============================] - 1s 390us/sample - loss: 0.6093 - accuracy: 0.6909
Epoch 819/1000
3601/3601 [==============================] - 3s 786us/sample - loss: 0.6076 - accuracy: 0.6895
Epoch 820/1000
3601/3601 [==============================] - 3s 875us/sample - loss: 0.6098 - accuracy: 0.6881
Epoch 821/1000
3601/3601 [==============================] - 3s 947us/sample - loss: 0.6073 - accuracy: 0.6884
Epoch 822/1000
3601/3601 [==============================] - 3s 744us/sample - loss: 0.6052 - accuracy: 0.6912
Epoch 823/1000
3601/3601 [==============================] - 2s 435us/sample - loss: 0.6093 - accuracy: 0.6884
Epoch 824/1000
3601/3601 [==============================] - 2s 417us/sample - loss: 0.6087 - accuracy: 0.6879
Epoch 825/1000
3601/3601 

3601/3601 [==============================] - 2s 462us/sample - loss: 0.6089 - accuracy: 0.6868
Epoch 965/1000
3601/3601 [==============================] - 2s 423us/sample - loss: 0.6102 - accuracy: 0.6881
Epoch 966/1000
3601/3601 [==============================] - 2s 454us/sample - loss: 0.6111 - accuracy: 0.6845
Epoch 967/1000
3601/3601 [==============================] - 2s 547us/sample - loss: 0.6104 - accuracy: 0.6895
Epoch 968/1000
3601/3601 [==============================] - 1s 348us/sample - loss: 0.6084 - accuracy: 0.6868
Epoch 969/1000
3601/3601 [==============================] - 1s 301us/sample - loss: 0.6080 - accuracy: 0.6876
Epoch 970/1000
3601/3601 [==============================] - 1s 322us/sample - loss: 0.6075 - accuracy: 0.6856
Epoch 971/1000
3601/3601 [==============================] - 1s 326us/sample - loss: 0.6085 - accuracy: 0.6884
Epoch 972/1000
3601/3601 [==============================] - 1s 328us/sample - loss: 0.6077 - accuracy: 0.6887
Epoch 973/1000
3601/3601 

[0.67702556 0.69256383 0.67777777 0.6807564  0.6807564 ]
Mean Accuracy: 0.6817759
Standard Deviation of the mean: 0.005603733


In [6]:
scores = np.array(scores)
print(scores)
print("Mean Accuracy:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[0.67702556 0.69256383 0.67777777 0.6807564  0.6807564 ]
Mean Accuracy: 0.6817759
Standard Deviation of the mean: 0.005603733
